In [13]:
import sys
sys.path.append("../")

In [14]:
import os.path as op
import random

import pandas as pd
import yaml

from agents import GreedyAgent,RandomAgent
from agents.matchmanager import MatchManager
from agents.ml.classifier import ClassifierAgent
from agents.ml.regression import RegressionAgent
from agents.ml.regressionMulti import RegressionMultiAgent
from core.const import RED, BLUE
from scenarios import scenarioJunction
import warnings

from multiprocessing import Pool

from itertools import product
import json

import os




In [15]:
with open(op.join('.', '../logger.config.yaml'), 'r') as stream:
    config = yaml.load(stream, Loader=yaml.FullLoader)
# logging.config.dictConfig(config)

In [16]:
def player(team:str, data:tuple, seed:int):
    if data[0] == 'gre':
        #print(f'{team:5}-> GreedyAgent')
        return GreedyAgent(team, seed=seed)
    if data[0] == 'cls':
        #print(f'{team:5}-> ClassifierAgent')
        return ClassifierAgent(team, data[1], seed=seed)
    if data[0] == 'reg':
        #print(f'{team:5}-> RegressionAgent')
        return RegressionAgent(team, data[1], seed=seed)
    if data[0]=='ran':
        #print(f'{team:5}-> RandomAgent')
        return RandomAgent(team, seed=seed)
    if data[0]=='regMulti':
        #print(f'{team:5}-> RegressionAgentMulti')
        return RegressionMultiAgent(team,data[1],data[2],data[3],seed=seed)


    
    return None

In [17]:
def game(args):
    scenario,seed,agentRed,agentBlue=args

    board, state = scenario()
    
    #print('seed:   ', seed)
    
    playerRed = player(RED, agentRed, seed)
    playerBlue = player(BLUE, agentBlue, seed)

    mm = MatchManager(' ', playerRed, playerBlue, board, state, seed=seed)
    while not mm.end:
        mm.nextStep()
    save={
        'scenario':board.name,
        'seed':seed,
        'teamRed':playerRed.name,
        'teamBlue':playerBlue.name,
        'winner':mm.winner
    }
    file_name=f'../output/{playerRed.name}_{playerBlue.name}_{board.name}_{seed}.json'
    #print(file_name)
    with open(file_name, 'w') as json_file:
        json.dump(save, json_file)
        #salva le cose su disco!
    
    #print('winner: ', mm.winner)

    #return mm

In [18]:
red_models = [
    ('gre', ''),
    ('ran', ''),
    #('cls', '../models/Junction_RandomForestClassifier_red_20210227.joblib'),
    ('cls', '../models/Junction_RandomForestClassifier_20210227.joblib'),
    #('reg', '../models/Junction_RandomForestRegressor_red_20210227.joblib'),
    ('reg', '../models/Junction_RandomForestRegressor_20210227.joblib'),
    ('regMulti','../models/Junction_RandomForestRegressor_red_attack_20210227.joblib','../models/Junction_RandomForestRegressor_red_move_20210227.joblib','../models/Junction_RandomForestRegressor_red_pass_20210227.joblib')

]
blue_models = [
    ('gre', ''),
    ('ran', ''),
    #('cls', '../models/Junction_RandomForestClassifier_blue_2021027.joblib'),
    ('cls', '../models/Junction_RandomForestClassifier_20210227.joblib'),
    #('reg', '../models/Junction_RandomForestRegressor_blue_20210227.joblib'),
    ('reg', '../models/Junction_RandomForestRegressor_20210227.joblib'),
    ('regMulti','../models/Junction_RandomForestRegressor_blue_attack_20210227.joblib','../models/Junction_RandomForestRegressor_blue_move_20210227.joblib','../models/Junction_RandomForestRegressor_blue_pass_20210227.joblib')


]

args=[]
scenarios = [scenarioJunction]

for i in range(0,1):
    for scenario in scenarios:
        s = random.randint(0, 1000000)
        r = random.randint(0, len(red_models)-1)
        b = random.randint(0, len(blue_models)-1)
        args.append((scenario,s,red_models[r],blue_models[b]))

In [12]:
for agentRed, agentBlue in product(red_models, blue_models):
    print(agentRed,agentBlue)


('gre', '') ('gre', '')
('gre', '') ('ran', '')
('gre', '') ('cls', '../models/Junction_RandomForestClassifier_20210227.joblib')
('gre', '') ('reg', '../models/Junction_RandomForestRegressor_20210227.joblib')
('ran', '') ('gre', '')
('ran', '') ('ran', '')
('ran', '') ('cls', '../models/Junction_RandomForestClassifier_20210227.joblib')
('ran', '') ('reg', '../models/Junction_RandomForestRegressor_20210227.joblib')
('cls', '../models/Junction_RandomForestClassifier_20210227.joblib') ('gre', '')
('cls', '../models/Junction_RandomForestClassifier_20210227.joblib') ('ran', '')
('cls', '../models/Junction_RandomForestClassifier_20210227.joblib') ('cls', '../models/Junction_RandomForestClassifier_20210227.joblib')
('cls', '../models/Junction_RandomForestClassifier_20210227.joblib') ('reg', '../models/Junction_RandomForestRegressor_20210227.joblib')
('reg', '../models/Junction_RandomForestRegressor_20210227.joblib') ('gre', '')
('reg', '../models/Junction_RandomForestRegressor_20210227.joblib

with Pool(64,maxtasksperchild=10) as p:
    p.map(game, args)

In [19]:
scenarios = [scenarioJunction]
args=[]
for i in range(0, 100):
    for scenario in scenarios:
        for agentRed, agentBlue in product(red_models, blue_models):
            args.append((scenario,i,agentRed,agentBlue))

In [ ]:
with Pool(64) as p:
    p.map(game, args)

Entropy out of range: 1.0330076353854272
Entropy out of range: 1.028173128262458
[-0.34, 0.66, 0.64, 0.64, 0.66, 0.64, 0.66, 0.64, 0.64, 0.72, 0.66, 0.68, 0.66, 0.68, 0.64, 0.64, 0.64, 0.66, 0.64, 0.64, 0.64, 0.64, 0.64, 0.66, 0.66, 0.68, 0.66, 0.64, 0.66, 0.68, 0.64, 0.64, 0.66, 0.66, 0.68, 0.66, 0.64, 0.68, 0.64, 0.64, 0.64, 0.66, 0.68, 0.68, 0.68, 0.64, 0.68, 0.64, 0.68, 0.64, 0.66, 0.68, 0.66, 0.64, 0.66, 0.68, 0.66, 0.64, 0.64, 0.66, 0.64, 0.64, 0.64, 0.68, 0.64, 0.66, 0.66, 0.64, 0.66, 0.66, 0.64, 0.64, 0.64, 0.64, 0.64, 0.68, 0.64, 0.64, -0.34, 0.64, 0.64, 0.64, 0.64, 0.64, 0.64, 0.64, 0.64, 0.64, 0.64, 0.64, 0.64, 0.64, 0.64, 0.64, 0.64, 0.64, 0.64, 0.64, 0.64, 0.64, 0.64, 0.64, 0.64, 0.64, 0.64, 0.64, 0.64, 0.64, 0.64, 0.64, 0.64, 0.64, 0.64, 0.64, 0.64, -0.34, 0.64, 0.64, 0.64, 0.64, 0.64, 0.64, 0.62, 0.64, 0.64, 0.64, 0.64, 0.62, 0.56, 0.64, 0.64, 0.64, 0.64, 0.62, 0.64, 0.64, 0.64, 0.64, 0.56, 0.62, 0.64, 0.64, 0.64, 0.56, 0.64, 0.64, 0.64, 0.62, 0.56, 0.64, 0.64]
[-0.24, 0